## Multi-omics Clustering

In [2]:
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(NMF))

### Data Preparation

In [3]:
PATH_DATA <- "../data_linkedomics/"

valid_samples <- read.csv("sample105.list", header = FALSE, col.names = "ID")
valid_samples$ID2 <- gsub("-", ".", valid_samples$ID) 
head(valid_samples,3)

,ID,ID2
,<chr>,<chr>
1,C3L-02613,C3L.02613
2,C3L-04072,C3L.04072
3,C3N-04282,C3N.04282


In [4]:
cnv <- read.table(paste0(PATH_DATA, "SCNA_log2_gene_level.cct"), header = TRUE, row.names = 1, sep = "\t", 
    # na.strings = "NA" 
    )
cnv <- cnv %>%
    # Select only the valid sample columns in order
    select(all_of(valid_samples$ID2)) %>%
    # Remove rows where any value is "NA"
    filter(rowSums(is.na(.)) == 0) %>%
    # Subtract median from each row
    mutate(row_median = apply(., 1, median, na.rm = TRUE)) %>%
    mutate(across(-row_median, ~ . - row_median)) %>%
    select(-row_median)
tail(cnv, 3)
dim(cnv)

,C3L.02613,C3L.04072,C3N.04282,C3L.03639,C3L.01328,C3L.00277,C3L.02604,C3L.01971,C3N.01167,C3L.01031,⋯,C3N.00436,C3N.02944,C3L.03356,C3N.02589,C3L.03388,C3N.01388,C3N.00198,C3N.01380,C3N.01715,C3N.03426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DAZ4,-0.12735,-0.59551,-0.92239,-0.72322,-0.02908,0.07958,0.82266,0.23247,-0.45531,0.28581,⋯,-0.36377,0.30116,-0.06191,-0.05711,-0.38335,-0.12518,-0.10954,0.11612,0.40891,-0.47479
BPY2C,-0.12735,-0.59551,-0.92239,-0.72322,-0.02908,0.07958,0.82266,0.23247,-0.45531,0.28581,⋯,-0.36377,0.30116,-0.06191,-0.05711,-0.38335,-0.12518,-0.10954,0.11612,0.40891,-0.47479
CDY1,-0.12735,-0.59551,-0.92239,-0.72322,-0.02908,0.07958,0.82266,0.23247,-0.45531,0.28581,⋯,-0.36377,0.30116,-0.06191,-0.05711,-0.38335,-0.12518,-0.10954,0.11612,0.40891,-0.47479


[1] 19855   105

In [5]:
prot <- read.table(paste0(PATH_DATA, "proteomics_gene_level_MD_abundance_tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
prot <- prot %>%
    # Order columns as in valid_samples
    select(valid_samples$ID2) %>%
    # sort the columns by the order in valid_samples
    select(order(match(names(.), valid_samples$ID2))) %>%
    # Remove rows with all NAs
    filter(rowSums(is.na(.)) == 0) %>%
    # median normalization
    mutate(row_median = apply(., 1, median, na.rm = TRUE)) %>%
    mutate(across(-row_median, ~ . / row_median)) %>%
    select(-row_median) %>%
    # Do log2 transformation
    mutate(across(everything(), ~ log2(.)))

head(prot, 3)
dim(prot)

,C3L.02613,C3L.04072,C3N.04282,C3L.03639,C3L.01328,C3L.00277,C3L.02604,C3L.01971,C3N.01167,C3L.01031,⋯,C3N.00436,C3N.02944,C3L.03356,C3N.02589,C3L.03388,C3N.01388,C3N.00198,C3N.01380,C3N.01715,C3N.03426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,-0.02990673,-0.05704551,0.01628874,0.01603432,-0.007786107,0.08029986,-0.002396511,-0.02007117,0.002849704,-0.012854305,⋯,-0.047168575,0.002205547,0.008227415,0.023093964,-0.01740784,0.007965865,0.006648201,0.03795399,0.0008283693,0.01275042
A1CF,-0.10768810,-0.06315414,-0.02110286,-0.03337825,-0.027146384,0.01286132,-0.043795955,0.04575079,0.065486572,-0.003501735,⋯,-0.007676754,0.034893213,0.001121374,0.014860119,-0.01008405,-0.049558683,0.050916236,-0.03390004,-0.0403040175,0.02859428
A2M,-0.04459164,-0.07561623,-0.01808992,-0.03178421,-0.005866177,0.02693054,0.006563652,0.03762665,0.006559212,-0.015757502,⋯,-0.016670852,0.000000000,0.052783243,0.004635516,-0.02938669,0.028271871,0.038905775,0.03043543,-0.0143345714,0.03923146


[1] 5773  105

In [6]:
rna <- read.table(paste0(PATH_DATA, "mRNA_RSEM_UQ_log2_Tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
rna <- rna %>%
    # Order columns as in valid_samples
    select(valid_samples$ID2) %>%
    # sort the columns by the order in valid_samples
    select(order(match(names(.), valid_samples$ID2))) %>%
    # Remove rows with any NAs or any zero values
    filter(rowSums(is.na(.)) == 0) %>%
    filter(rowSums(. == 0) == 0) %>%
    # Subtract the median of each row 
    mutate(row_median = apply(., 1, median, na.rm = TRUE)) %>%
    mutate(across(-row_median, ~ . - row_median)) %>%
    select(-row_median)

dim(rna)
head(rna, 3)

[1] 18197   105

,C3L.02613,C3L.04072,C3N.04282,C3L.03639,C3L.01328,C3L.00277,C3L.02604,C3L.01971,C3N.01167,C3L.01031,⋯,C3N.00436,C3N.02944,C3L.03356,C3N.02589,C3L.03388,C3N.01388,C3N.00198,C3N.01380,C3N.01715,C3N.03426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,-0.6208639,-1.5641153,-0.03598678,-1.499229,-1.030409,-0.3632095,-0.7079235,0.4973275,0.1398404,0.4582964,⋯,0.5012384,0.1962640,0.45922795,0.05454697,-0.3385884,-0.06842248,0.5971399,0.72255331,0.04504042,-1.018051
A1BG-AS1,-0.8663854,-0.8302223,0.11105590,-1.422656,-1.161717,-1.8484433,-0.5696947,0.3464761,-0.2569541,0.4220428,⋯,0.3638638,0.3672824,0.31895969,-0.71812584,-0.2179410,-0.09720916,0.3269861,0.70061799,0.30224950,-1.071129
A1CF,-2.8039993,-3.0594026,-2.42468317,-1.982175,-1.343388,-2.0692887,-1.9460596,1.0109679,2.3566886,-0.2918026,⋯,0.2894103,0.9205599,0.06451494,0.61939774,-1.5979746,-2.31924199,1.5137978,-0.01529477,-2.96096702,1.411194


In [7]:
# glyco <- read.table(paste0(PATH_DATA, "N-glycoproteomics_peptide_level_ratio_tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
glyco <- read.table(paste0(PATH_DATA, "N-glycoproteomics_Site_level_ratio_tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
glyco <- glyco %>%
    # Order columns as in valid_samples
    select(valid_samples$ID2) %>%
    # sort the columns by the order in valid_samples
    select(order(match(names(.), valid_samples$ID2))) %>%
    # Remove rows with all NAs
    filter(rowSums(is.na(.)) == 0) #%>%
    # Subtract the median of each row 
    # mutate(across(everything(), ~ . - median(., na.rm = TRUE)))

dim(glyco)
head(glyco, 3)

[1] 1454  105

,C3L.02613,C3L.04072,C3N.04282,C3L.03639,C3L.01328,C3L.00277,C3L.02604,C3L.01971,C3N.01167,C3L.01031,⋯,C3N.00436,C3N.02944,C3L.03356,C3N.02589,C3L.03388,C3N.01388,C3N.00198,C3N.01380,C3N.01715,C3N.03426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n[TMT11plex]RHEEGHMLNC[Carbamidomethyl]TC[Carbamidomethyl]FGQGR-N3H4F0S0G0,0.2711286,0.3086638,1.1849827,-0.006491598,1.9091718,1.278178,1.0885962,-0.7653889,-0.9867006,-0.5941954,⋯,0.57288470,0.9236759,-0.5551570,-0.24003358,-0.2509592,0.05717399,-0.16628917,-0.5030644,0.7816165,1.034915863
n[TMT11plex]LLQVVYLHSNNITK[TMT11plex]-N5H6F1S2G0,-0.1230223,-0.4341817,0.2447100,0.079600075,-0.9919879,1.089329,-0.1866849,0.3223009,0.4387361,0.2199116,⋯,-0.94928102,-1.3285715,1.3734742,0.73234040,-1.2241213,-0.83878453,-0.08174441,0.7487903,0.1999972,-0.779378985
n[TMT11plex]NYTADYDK[TMT11plex]-N2H8F0S0G0,0.5089437,0.5752988,-0.2466641,-0.213866510,0.1689218,-1.305813,-0.3507612,0.3861138,-0.9076605,-0.2210551,⋯,0.08706562,-0.9358628,-0.7485728,-0.04754478,0.6560518,-0.76525368,-1.11018380,-0.9045606,-0.2454482,0.006003751


In [8]:
# psty <- read.table(paste0(PATH_DATA, "phosphoproteomics_gene_level_MD_abundance_tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
psty <- read.table(paste0(PATH_DATA, "phosphoproteomics_site_level_MD_abundance_tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
# psty <- read.table(paste0(PATH_DATA, "phosphoproteomics_MultiSite_level_MD_abundance_tumor.cct"), header = TRUE, row.names = 1, sep = "\t", na.strings = "NA")
psty <- psty %>%
    # Only select columns that are in valid_samples
    select(valid_samples$ID2) %>%
    # sort the columns by the order in valid_samples
    select(order(match(names(.), valid_samples$ID2))) %>%
    # Remove rows with all NAs or too small values
    filter(rowSums(is.na(.)) == 0) %>%
    filter(rowSums(. < 1e-10) == 0) %>%
    # Subtract the median of each row 
    mutate(row_median = apply(., 1, median, na.rm = TRUE)) %>%
    mutate(across(-row_median, ~ . / row_median)) %>%
    select(-row_median) %>%
    # Do log2 transformation
    # mutate(across(everything(), ~ log2(pmax(., 1e-10))))
    mutate(across(everything(), ~ log2(.)))
dim(psty)
head(psty, 3)

[1] 2037  105

,C3L.02613,C3L.04072,C3N.04282,C3L.03639,C3L.01328,C3L.00277,C3L.02604,C3L.01971,C3N.01167,C3L.01031,⋯,C3N.00436,C3N.02944,C3L.03356,C3N.02589,C3L.03388,C3N.01388,C3N.00198,C3N.01380,C3N.01715,C3N.03426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NP_000028.3_S781,-0.003289252,-0.098820469,-0.001863032,0.06418698,-0.01615852,-0.08553476,0.06142474,0.09203510,-0.002218144,-0.02439635,⋯,0.05133770,0.07273598,0.062203354,-0.064013021,-0.019261737,0.282415071,0.08484637,0.04799679,0.15066791,0.2849448
NP_000028.3_S834,-0.080017563,0.000000000,-0.034027167,0.03720242,-0.01953778,-0.03624982,0.03503582,0.09724809,-0.008214741,-0.02389460,⋯,0.01015709,0.03829329,0.031775950,-0.037113360,-0.006308129,0.200502184,0.03429170,0.03052564,0.14200041,0.1927270
NP_000072.2_S2105,-0.020938229,-0.009456211,-0.024632151,0.01100132,-0.06223903,-0.02472232,-0.02370921,0.02074372,0.003174092,0.03309474,⋯,-0.14639489,-0.01789628,0.007726258,0.004582555,-0.002117515,-0.009992859,0.01682114,0.00000000,-0.01286737,-0.1006145


In [9]:
# Concatenate all data frames
data <- bind_rows(
    cnv = cnv,
    prot = prot,
    rna = rna,
    glyco = glyco,
    psty = psty,
    .id = "data_type"
)
dim(data)
row_sd <- apply(data[ , -1], 1, sd, na.rm = TRUE)
cutoff <- quantile(row_sd, probs = 0.05, na.rm = TRUE)
data <- data[row_sd > cutoff, ] %>%
    # Column-wise z-score normalization
    mutate(across(-data_type, ~ (.-mean(.)) / sd(.))) %>%
    # Sort columns by name
    select(data_type, sort(names(.)[-1]))

dim(data)
tail(data,10)
# Count the number of features per data type
feature_counts <- data %>%
    group_by(data_type) %>%
    summarise(feature_count = n()) %>%
    ungroup()
print(feature_counts)

[1] 47316   106

[1] 44950   106

,data_type,C3L.00017,C3L.00102,C3L.00277,C3L.00589,C3L.00598,C3L.00599,C3L.00622,C3L.00625,C3L.00819,⋯,C3N.03754,C3N.03780,C3N.03839,C3N.03840,C3N.03853,C3N.03884,C3N.04119,C3N.04126,C3N.04282,C3N.04283
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NP_958845.1_S79,psty,0.079446002,0.03630348,-0.026318794,-0.013611849,-0.05005705,-0.172905292,-0.088639119,-0.050904401,-0.007752476,⋯,0.021672913,0.0047389330,0.1707685412,-0.10959935,0.050341969,0.08423931,-0.059991279,-0.0007296711,0.06338611,0.0100217780
NP_976324.2_S575,psty,0.082564705,0.02576112,0.068521890,-0.039887376,0.10901623,0.170325698,-0.131091317,0.053230996,0.053485233,⋯,0.008844694,-0.0448768665,0.0792314601,-0.18174744,0.009397321,0.11084302,-0.060311673,-0.0932079326,0.07073944,0.0365422279
NP_976324.2_S710,psty,0.147034528,0.01044562,0.111215602,-0.035994889,0.07925124,0.063384138,-0.069736553,0.088925435,0.005206951,⋯,-0.016275165,0.0003539852,0.0787260371,-0.15433375,0.060727164,0.11275161,-0.014349525,-0.0694289794,0.06985248,0.0586667396
NP_982272.2_S904,psty,0.023798552,0.01650842,-0.094195741,-0.041668947,-0.07598812,-0.149207133,-0.068073138,-0.159080895,-0.029022903,⋯,0.019838845,-0.0850848303,0.0614341098,-0.01065424,0.061241222,0.13993751,-0.049799194,-0.1248938067,0.10080156,0.0002623087
NP_987100.1_T227,psty,0.146972631,-0.44806034,0.212148654,0.120822055,-0.02753782,0.222282194,-0.267095711,-0.088231852,0.001989650,⋯,-0.128095569,0.0814034202,-0.0004391136,0.12351415,0.160735470,0.02179675,-0.215216260,0.1531561797,0.16853110,-0.0118980910
NP_996826.2_S1414,psty,0.063367469,0.10591795,0.153282608,0.045718296,-0.15663787,-0.060666862,0.007299217,0.193374219,0.095686560,⋯,-0.006382670,-0.0651741198,0.0346258925,0.01082142,-0.012965689,0.06419983,-0.004291478,0.0680142954,0.07381013,0.0332533510
NP_997245.2_S881,psty,0.137718794,-0.04078144,0.004700958,0.130736559,-0.06173706,0.006313795,0.090414648,-0.001235223,-0.086052319,⋯,-0.157811497,0.0578804281,0.0710230697,0.07991204,0.012761431,0.06018773,0.045825368,0.0684361940,0.04752730,-0.0297662448
NP_997641.1_S2294,psty,0.025348876,0.06201465,0.281078580,-0.007967676,0.03825847,-0.105194314,-0.133995231,0.213056396,0.043122006,⋯,-0.106894553,-0.0390407275,-0.0787479171,0.07213424,0.088405738,-0.01837157,0.018400899,0.0287166604,0.13896632,0.1326827699
NP_998754.1_S1154,psty,-0.003492166,-0.01452163,0.065315431,-0.051250769,0.10283842,-0.032190435,-0.061569401,-0.151963153,0.022449098,⋯,0.038036967,0.0080910620,0.1052848222,0.09795117,0.113101530,-0.00430832,-0.022071470,-0.0709854020,0.12219448,0.1323997326


# A tibble: 5 × 2
  data_type feature_count
  <chr>             <int>
1 cnv               19855
2 glyco              1454
3 prot               3490
4 psty               1954
5 rna               18197


In [10]:
data_type <- data$data_type
data_num <- data[, -1]

positive_only_matrix <- data_num
positive_only_matrix[positive_only_matrix < 0] <- 0
# Abs values of negative values
negative_only_matrix <- data_num
negative_only_matrix[negative_only_matrix > 0] <- 0
negative_only_matrix <- abs(negative_only_matrix)

combined_matrix <- rbind(positive_only_matrix, negative_only_matrix)

combined_matrix <- combined_matrix[rowSums(combined_matrix) != 0, ]

dim(data)
dim(combined_matrix)

[1] 44950   106

[1] 89900   105

In [ ]:
membership <- nmf_clu@fit@H %>%
    t() %>%
    as.data.frame() %>%
    # Set the cluster column, V1 > V2
    mutate(cluster = ifelse(V1 >= V2, "1", "2")) %>%
    # Divide by the sum of the two 
    mutate(sum = V1 + V2) %>%
    mutate(V1 = V1 / sum, V2 = V2 / sum) %>%
    mutate(cluster = ifelse(V1 >= V2, "1", "2"))
membership

,V1,V2,cluster,newsum
,<dbl>,<dbl>,<chr>,<dbl>
C3L.00017,2.321938e-01,0.76780622,2,1
C3L.00102,4.576865e-01,0.54231351,2,1
C3L.00277,6.720580e-01,0.32794204,1,1
C3L.00589,1.816791e-01,0.81832090,2,1
C3L.00598,9.131920e-01,0.08680796,1,1
C3L.00599,5.268393e-02,0.94731607,2,1
C3L.00622,2.099518e-01,0.79004823,2,1
C3L.00625,3.786299e-01,0.62137005,2,1
C3L.00819,7.698312e-01,0.23016879,1,1


In [ ]:
nmf_clu <- nmf(as.matrix(combined_matrix), 2, .options=list(verbose = TRUE), maxIter=500)
membership <- nmf_clu@fit@H %>%
    t() %>%
    as.data.frame() %>%
    # Set the cluster column, V1 > V2
    mutate(cluster = ifelse(V1 >= V2, "1", "2")) %>%
    # Divide by the sum of the two 
    mutate(sum = V1 + V2) %>%
    mutate(V1 = V1 / sum, V2 = V2 / sum) %>%
    select(-sum) %>%
    mutate(cluster = ifelse(V1 >= V2, "1", "2"))

# Count the number of samples in each cluster
membership_summary <- membership %>%
    group_by(cluster) %>%
    summarise(count = n())
print(membership_summary)


write.table(membership, "membership.tsv", sep = "\t", quote = FALSE)

NMF algorithm: 'brunet'

NMF seeding method: random



Iterations: 500/500 
DONE (stopped at 500/500 iterations)
# A tibble: 2 × 2
  cluster count
  <chr>   <int>
1 1          53
2 2          52


ERROR: Error in rownames_to_column(., "ID"): could not find function "rownames_to_column"


In [21]:
# Replace "-" with "." in row names
membership <- membership %>%
    mutate(ID = gsub("\\.", "-", ID)) %>%
    column_to_rownames("ID")
membership



ERROR: Error in column_to_rownames(., "ID"): could not find function "column_to_rownames"
